In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

train = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_v0.0.5.csv')
test = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_v0.0.5.csv')

In [3]:
def transform_row(r):
    return '{entity1} {org_path} {entity2}'.format(entity1=r.type1[0:3], org_path=r.path, entity2=r.type2[0:3])

train['path'] = train.apply(transform_row, axis=1)
test['path'] = test.apply(transform_row, axis=1)

In [4]:
relation_to_paths = train.groupby(['relation'])['path'].unique().to_dict()

In [5]:
def label_sentence_recall(test_row, relation):
    path = test_row['path']
    true_relation = test_row['relation']
    
    if path in relation_to_paths[relation]:
        if true_relation == relation:
            return 'TP'
        else:
            return 'FP'
    else:
        return 'N'

In [6]:
for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):
    test[relation] = test.apply(lambda row : label_sentence_recall(row, relation), axis=1)    

In [7]:
stats = []
overall_tp = 0
overall_fp = 0

for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths.keys()):

    counts = test[relation].value_counts().to_dict()
        
    tp = counts.get('TP', 0)
    fp = counts.get('FP', 0)
    precision = tp / (tp+fp)

    stats.append( (relation, round(precision,4)) )
    
    overall_tp += tp
    overall_fp += fp
    
overall_precision = overall_tp / (overall_tp+overall_fp)

    

stats.append( ('overall', round(overall_precision,4)) )
stats = sorted(stats, key=lambda x: x[1], reverse=True)

In [12]:
import plotly.graph_objs as go

overall_index = next(i for (i, (relation, score)) in enumerate(stats) if relation == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'


trace1 = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Precision',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace1]
layout = go.Layout(
    title='UD Path with Entity Type - Precision',
    xaxis_title="Relation",
    yaxis_title="Precision ",        
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [10]:
stats

[('per:charges', 1.0),
 ('per:religion', 1.0),
 ('org:website', 0.9091),
 ('per:age', 0.8429),
 ('per:title', 0.8167),
 ('org:city_of_headquarters', 0.6977),
 ('org:country_of_headquarters', 0.6452),
 ('per:origin', 0.6119),
 ('per:cause_of_death', 0.6),
 ('org:alternate_names', 0.5585),
 ('per:cities_of_residence', 0.5556),
 ('org:top_members/employees', 0.5521),
 ('org:stateorprovince_of_headquarters', 0.55),
 ('per:employee_of', 0.4332),
 ('per:stateorprovinces_of_residence', 0.4),
 ('per:countries_of_residence', 0.3492),
 ('org:number_of_employees/members', 0.3333),
 ('org:founded', 0.3182),
 ('overall', 0.2521),
 ('per:stateorprovince_of_birth', 0.25),
 ('per:city_of_death', 0.1875),
 ('org:political/religious_affiliation', 0.1765),
 ('org:founded_by', 0.1633),
 ('per:stateorprovince_of_death', 0.1538),
 ('per:schools_attended', 0.1489),
 ('per:parents', 0.1309),
 ('per:date_of_birth', 0.1163),
 ('per:city_of_birth', 0.1111),
 ('per:date_of_death', 0.0909),
 ('org:subsidiaries', 0